In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.offline as py
import requests
import json
from geopy.geocoders import Nominatim
from config import gkey

In [23]:
# File to load
csv_crime_2012 = "Table_13_Hate_Crime_Incidents_per_Bias_Motivation_and_Quarter_by_State_and_Agency_2012.csv"
csv_crime_2013 = "Table_13_Hate_Crime_Incidents_per_Bias_Motivation_and_Quarter_by_State_and_Agency_2013.csv"
csv_crime_2014 = "Table_13_Hate_Crime_Incidents_per_Bias_Motivation_and_Quarter_by_State_and_Agency_2014.csv"
csv_crime_2015 = "Table_13_Hate_Crime_Incidents_per_Bias_Motivation_and_Quarter_by_State_and_Agency_2015.csv"
csv_crime_2016 = "Table_13_Hate_Crime_Incidents_per_Bias_Motivation_and_Quarter_by_State_and_Agency_2016.csv"
csv_states = "states.csv"
csv_population = "population_estimates.csv"

states_df = pd.read_csv(csv_states)
population_df = pd.read_csv(csv_population)
hate_crime_2012 = pd.read_csv(csv_crime_2012)
hate_crime_2013 = pd.read_csv(csv_crime_2013)
hate_crime_2014 = pd.read_csv(csv_crime_2014)
hate_crime_2015 = pd.read_csv(csv_crime_2015)
hate_crime_2016 = pd.read_csv(csv_crime_2016)

hate_crime_2012['2012 Crimes'] = hate_crime_2012['Race'] + hate_crime_2012['Ethnicity']
hate_crime_2013['2013 Crimes'] = hate_crime_2013['Race'] + hate_crime_2013['Ethnicity']
hate_crime_2014['2014 Crimes'] = hate_crime_2014['Race'] + hate_crime_2014['Ethnicity']
hate_crime_2015 = hate_crime_2015.rename(columns={'Race/\nEthnicity/\nAncestry': '2015 Crimes'})
hate_crime_2016 = hate_crime_2016.rename(columns={'Race/\nEthnicity/\nAncestry': '2016 Crimes'})

hate_crime_2012 = hate_crime_2012[["State", "Agency name", "2012 Crimes", "Population"]]
hate_crime_2013 = hate_crime_2013[["State", "Agency name", "2013 Crimes", "Population"]]
hate_crime_2014 = hate_crime_2014[["State", "Agency name", "2014 Crimes", "Population"]]
hate_crime_2015 = hate_crime_2015[["State", "Agency name", "2015 Crimes", "Population"]]
hate_crime_2016 = hate_crime_2016[["State", "Agency name", "2016 Crimes", "Population"]]

hate_crime_2012 = hate_crime_2012[hate_crime_2012['Agency name'].notnull()]
hate_crime_2012 = hate_crime_2012[hate_crime_2012['Population'].notnull()]
hate_crime_2013 = hate_crime_2013[hate_crime_2013['Agency name'].notnull()]
hate_crime_2013 = hate_crime_2013[hate_crime_2013['Population'].notnull()]
hate_crime_2014 = hate_crime_2014[hate_crime_2014['Agency name'].notnull()]
hate_crime_2014 = hate_crime_2014[hate_crime_2014['Population'].notnull()]
hate_crime_2015 = hate_crime_2015[hate_crime_2015['Agency name'].notnull()]
hate_crime_2015 = hate_crime_2015[hate_crime_2015['Population'].notnull()]
hate_crime_2016 = hate_crime_2016[hate_crime_2016['Agency name'].notnull()]
hate_crime_2016 = hate_crime_2016[hate_crime_2016['Population'].notnull()]



In [24]:

counties_2012 = hate_crime_2012[["State", "Agency name"]]
counties_2013 = hate_crime_2013[["State", "Agency name"]]
counties_2014 = hate_crime_2014[["State", "Agency name"]]
counties_2015 = hate_crime_2015[["State", "Agency name"]]
counties_2016 = hate_crime_2016[["State", "Agency name"]]

counties_df = pd.merge(counties_2012, counties_2013, on=["State","Agency name"], how="outer")
counties_df = pd.merge(counties_df, counties_2014, on=["State","Agency name"], how="outer")
counties_df = pd.merge(counties_df, counties_2015, on=["State","Agency name"], how="outer")
counties_df = pd.merge(counties_df, counties_2016, on=["State","Agency name"], how="outer")
counties_df = counties_df[counties_df['Agency name'].notnull()]
counties_df = counties_df.drop_duplicates()
counties_df = counties_df.dropna(subset=['Agency name'])

counties_df = counties_df.sort_values(by=['State', 'Agency name'])

#counties_df = counties_df.set_index(['State'])
#counties_df = counties_df.groupby("State")
counties_df = counties_df.loc[counties_df['State'] == 'Alaska']

counties_df.head(500)
#hate_crime_df['Agency name'].value_counts()

,State,Agency name
1,Alaska,Anchorage
3180,Alaska,Juneau
2813,Alaska,Kotzebue
2814,Alaska,North Pole
2815,Alaska,Soldotna
3181,Alaska,Wasilla


In [ ]:
for (idx, row) in counties_df.iterrows():
    try:
        agency = (row.loc['Agency name'])
        state = (row.loc['State'])
        query_url = f"https://maps.googleapis.com/maps/api/geocode/json?address={agency}&key={gkey}"
        response = requests.get(query_url).json()
        lat = response["results"][0]["geometry"]["location"]["lat"]
        lng = response["results"][0]["geometry"]["location"]["lng"]
        #counties_df['Latitude'] = lat
        #counties_df['Longitude'] = lng
        counties_df.at[idx, 'Latitude'] = lat
        counties_df.at[idx, 'Longitude'] = lng
        #print(f"{agency},{state}:{lat},{lng}")
    except:
        #pass
        print("continue")
        continue
counties_df.head()

In [ ]:
counties_df = pd.merge(counties_df, hate_crime_2012, on=["State","Agency name"], how="left")
counties_df = pd.merge(counties_df, hate_crime_2013, on=["State","Agency name"], how="left")
counties_df = pd.merge(counties_df, hate_crime_2014, on=["State","Agency name"], how="left")
counties_df = pd.merge(counties_df, hate_crime_2015, on=["State","Agency name"], how="left")
counties_df = pd.merge(counties_df, hate_crime_2016, on=["State","Agency name"], how="left")



counties_df.head()


In [ ]:
#Save DataFrame to CSV File
#counties_df.to_csv('hate_crime_by_state_2012-2016.csv')